In [48]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from tqdm import tqdm_notebook
import pandas as pd
import time
import datetime
import json


def yogiyo_crawling_store_info(address, food_category):

    list_url = "https://www.yogiyo.co.kr/mobile/#/"

    # 주소 기준으로 초기화
    #driver = webdriver.Chrome('chromedriver')
    driver = webdriver.Chrome('/Users/jijoonghong/Downloads/chromedriver')

    driver.get(list_url)
    time.sleep(4)
    element = driver.find_element_by_name("address_input")
    element.clear()
    element.send_keys(address)
    btn = driver.find_element_by_css_selector("#button_search_address > button.btn.btn-default.ico-pick")
    btn.click()
    time.sleep(1)

    # 리뷰 많은 순으로 sorting
    driver.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[2]/div/select').click()
    driver.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[2]/div/select/option[3]').click()
    driver.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[2]/div/select').click()
    time.sleep(1)

    # 초기 데이터 프레임 생성
    df = pd.DataFrame(columns=['가게명', '총평점', '리뷰개수', '사장님리뷰개수', '최소주문금액', '배달시간','배달료', 
                               '맛 평점', '양 평점', '배달 평점', '영업시간', '위치', '세스코유무'])

    # 해당 카테고리에 접속하여 마지막까지 스크롤
    driver.get(list_url + food_category)
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:

        scroll_down = 0
        while scroll_down < 10:
            element.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.2)
            scroll_down += 1

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("끝")
            break

        last_height = new_height
    
    # 모든 가게의 정보를 담고 있는 html 파싱 
    main_html = driver.page_source
    soup = BeautifulSoup(main_html, 'html.parser')
    
    #가게 이름 리스트
    store_name = soup.find_all(attrs={'class':'restaurant-name ng-binding'})
    store_name = list(map(lambda x: x.text, store_name))    
    
    #총평점 리스트
    star_review = soup.find_all(attrs={'ng-show':'restaurant.review_avg > 0'})
    star_review = list(map(lambda x: float(x.text[-3:]), star_review))  
    
    #리뷰개수 리스트 
    review_count = soup.find_all(attrs={'ng-show':'restaurant.review_count > 0'})
    review_count = list(map(lambda x: int(x.text[30:-23]), review_count))  
    
    #사장님 댓글 갯수 리스트
    owner_count = soup.find_all(attrs={'ng-show':'restaurant.owner_reply_count > 0'})
    owner_count = list(map(lambda x: int(x.text[33:-23]),owner_count))  

    #최소 주문 금액 리스트
    min_price = soup.find_all(attrs={'class':'min-price ng-binding'})
    min_price = list(map(lambda x: int(x.text[:-7].replace(',','')), min_price))  

    #배달 시간 리스트
    delivery_time = soup.find_all(attrs={'ng-show':'restaurant.estimated_delivery_time'})
    delivery_time = list(map(lambda x: x.text[25:-23], delivery_time))  
    
    df['가게명'] = store_name
    df['총평점'] = star_review
    df['리뷰개수'] = review_count
    df['사장님리뷰개수'] = owner_count
    df['최소주문금액'] = min_price
    df['배달시간'] = delivery_time
    
    ### 가게정보 파싱 끝 ###

    ### 메뉴 및 세부 가게정보 가게별로 파싱###
    df.set_index('가게명', inplace = True)

    dic3 = {} 
    df2 = pd.read_csv("./"+address.split(" ")[1]+"url/"+address.split(" ")[1]+"_"+food_category+"_세부업종.csv")
    urls = df2['url'].tolist()
    
    # 가게별로 순회
    for i in range(len(urls)):
        try:
            driver.get(urls[i])
                      
            # 메뉴 크롤링
            time.sleep(2)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            name = soup.find('span', attrs={'class' : "restaurant-name ng-binding"}).text
            sub_lists = soup.find_all('div', attrs={'class':'panel panel-default ng-scope', 'ng-repeat':"category in restaurant.menu"})
            sub_lists = sub_lists[1:]

            dic2 = {}
            for l in sub_lists:
                menu_category = l.find('span', attrs={'ng-class':'get_menu_class(category.slug)'}).text
                menus = l.find_all('td', attrs={'class' : 'menu-text'})
                for menu in menus:
                    menu_name = menu.find('div', attrs={'class' : 'menu-name ng-binding'}).text
                    menu_price = menu.find('span', attrs={'ng-bind' : 'item.price|krw'}).text
                    dic2[menu_name] = menu_price
            dic3[name] = dic2

            # 평점 크롤링
            review = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a')
            driver.execute_script("arguments[0].click();", review)
            time.sleep(1)
            taste_star = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[5]/div[1]/div/ul/li[1]/span[2]/span[6]').text
            df.loc[name,'맛 평점'] = taste_star
            quantity_star = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[5]/div[1]/div/ul/li[2]/span[2]/span[6]').text
            df.loc[name,'양 평점'] = quantity_star
            delivery_star = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[5]/div[1]/div/ul/li[3]/span[2]/span[6]').text
            df.loc[name,'배달 평점'] = delivery_star

            # 가게 세부 정보 크롤링
            info = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[3]/a')
            driver.execute_script("arguments[0].click();", info)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            infos = soup.find_all(attrs={'class':'info-item'})
            for info in infos:
                if "업체정보" in info.text:
                    try:
                        delivery_fee = soup.find('span', attrs={'class': "list-group-item clearfix text-right ng-binding"}).text 
                    except:
                        delivery_fee = 0
                      
                    print(delivery_fee)
                    df.loc[name,'배달료'] = delivery_fee
                      
                    is_cesco = 0 if info.find('p', attrs={'ng-show':'restaurant.tags.length > 0 && restaurant.tags.indexOf("CESCO") >= 0',
                                                          'class':'ng-hide'}) is not None else 1
                    print(is_cesco)
                    df.loc[name,'세스코유무'] = is_cesco
                    opening_hour = info.find('p').find('span').text if "영업시간" in info.find('p').text else ""
                    print(opening_hour)
                    df.loc[name,'영업시간'] = opening_hour
                    location = info.find('p', attrs={'ng-show':"restaurant.address.length > 0"}).find('span').text
                    print(location)
                    df.loc[name,'위치'] = location
                    break

            time.sleep(3)

        except:
            continue # url은 보유하고있으나 이후 가맹 해지된 업장 예외처리
    
    # json, csv 저장
    json_file = './'+ address.split(" ")[1]+'_'+food_category+'_메뉴정보.json'
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(dic3, f, ensure_ascii = False)
    
    df.to_csv('./'+ address.split(" ")[1]+'_'+food_category+'_가게정보.csv')

In [49]:
yogiyo_crawling_store_info('서울특별시 강남구 삼성동 16-1 강남구청', "족발보쌈")

끝

        배달요금 1,000원 별도 (0원 이상 주문시 배달무료)

1
10:00 - 02:00
서울 강남구 삼성동 5-1 1층8,9호

        배달요금 1,000원 별도 (0원 이상 주문시 배달무료)

0
11:00 - 04:00
서울 강남구 삼성동 37-13 삼성동좋은사람좋은집 1층

        배달요금 1,000원 별도 (0원 이상 주문시 배달무료)

1
11:00 - 05:00
서울 강남구 신사동 651-9 2층

        배달요금 3,000원 별도 (0원 이상 주문시 배달무료)

1
11:00 - 23:59
서울 강남구 역삼동 643-14 지하1층

        배달요금 1,000원 별도 (50,000원 이상 주문시 배달무료)

0
11:00 - 01:00
서울 강남구 삼성동 157-18 지하1층 제비101호 주방7-1호

        배달요금 3,000원 별도 (0원 이상 주문시 배달무료)

0
10:00 - 09:59
서울 강남구 대치동 896-27

        배달요금 1,000원 별도 (0원 이상 주문시 배달무료)

1
07:00 - 06:59
서울특별시 강남구 논현동 232-23 지하 1층

        배달요금 2,000원 별도 (0원 이상 주문시 배달무료)

1
휴무일
서울 강남구 역삼동 677 서울철강빌딩 1층

        배달요금 4,000원 별도 (0원 이상 주문시 배달무료)

1
15:00 - 05:00
서울 강남구 역삼동 678-1 1층 102호

        배달요금 1,000원 별도 (0원 이상 주문시 배달무료)

0
11:00 - 03:00
서울 강남구 역삼동 700-24 지상1층

        배달요금 3,000원 별도 (0원 이상 주문시 배달무료)

1
15:00 - 05:00
서울 강남구 논현동 116-3 2층 201호

        배달요금 3,000원 별도 (0원 이상 주문시 배달무료)

0
17:00 - 04:00
서울 강남구 삼성동 119

        배달요금 1,